https://map.naver.com/v5/?c=14138728.9902886,4517748.2116690,15,0,0,0,dh

# NAVER map review crawling




네이버 지도에서 장소를 선택했을 때 보이는 리뷰 크롤링 하기

1. 네이버 지도 검색어 기반 url 수집
2. url 접속하여 리뷰 크롤링

## 검색어 기반 url 수집

In [1]:
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException 
import time 
import re 
from bs4 import BeautifulSoup 
from tqdm import tqdm 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import warnings 
warnings.filterwarnings(action='ignore') 

In [2]:
keyword = '그릭요거트'

In [3]:
driver = webdriver.Chrome(executable_path=r'chromedriver.exe') 

try: 
    naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place' # 검색 url 만들기 

    driver.get(naver_map_search_url) #검색 url 접속
    time.sleep(4)
    
    driver.switch_to.frame("searchIframe") #프레임 전환

    #파싱
    html = driver.page_source 
    soup = BeautifulSoup(html, 'lxml') 
    time.sleep(1) 

    #장소 넘버가 태그가 아닌 script에 담겨있어서 소스을 text(str)로 보고 정규표현식으로 추출
    #1번째 페이지의 50개 url 정보 가져옴. 
    aaa = soup.html.body.text

    #숫자 추출
    p = re.compile('{}(.*?){}'.format(re.escape('"id":"RestaurantListSummary:'), re.escape('","typename"')))
    place_list = p.findall(aaa)

except : 
    pass

In [4]:
final_url = []
for i in range(50): #50 이후엔 다른 문자가 함께 들어와서 range를 직접 숫자로 지정 
    final_url.append('https://pcmap.place.naver.com/restaurant/'+place_list[i]+'/review/visitor#')   

In [5]:
url_df = pd.DataFrame(final_url, columns = ['url'])
url_df.head()

,url
0,https://pcmap.place.naver.com/restaurant/19515...
1,https://pcmap.place.naver.com/restaurant/18685...
2,https://pcmap.place.naver.com/restaurant/12816...
3,https://pcmap.place.naver.com/restaurant/84025...
4,https://pcmap.place.naver.com/restaurant/11514...


In [6]:
url_df.to_csv("naver_place_url.csv")

## 수집한 url 기반 리뷰 텍스트 크롤링

In [7]:
url_df = pd.read_csv("naver_place_url.csv")

In [16]:
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException 
import time 
import re 
from bs4 import BeautifulSoup 
from tqdm import tqdm 

# 웹드라이버 접속 
driver = webdriver.Chrome('chromedriver.exe') 

# 수집할 정보
review_list = [] # 리뷰 
url_list=[]
place_name=[]
user_list=[]

In [17]:
#for i in range(len(url_df)): 

for i in range(3): # ★샘플로 3개만. 실 사용시에는 위 주석 코드로 사용.
    # 식당 리뷰 개별 url 접속 
    driver.get(url_df['url'][i]) 
    thisurl = url_df['url'][i] 
    time.sleep(2) 
    
    # 더보기 버튼 끝까지 누르기 
    try :
        while driver.find_element(By.CLASS_NAME, "_3iTUo").is_displayed:
            try :
                time.sleep(3) #더보기 버튼을 너무 빨리 누르면 새로운 리뷰가 뜨지 않고 기존 페이지 리뷰가 반복해서 뜸 #여기 때문에 느림
                WebDriverWait(driver,2).until(EC.element_to_be_clickable((By.CLASS_NAME, '_3iTUo'))).click()
            except TimeoutException :
                break
    except :
        pass

    # 파싱 
    time.sleep(1) 
    html = driver.page_source 
    soup = BeautifulSoup(html, 'lxml') 
    time.sleep(1) 
    
    # title
    title = soup.find('span', attrs = {'class':'_3XamX'}).text

    # 리뷰 텍스트 -> 공백 발생 O
    num = soup.find_all('li', attrs = {'class':'_3l2Wz'})
    #reviews = soup.find_all('span', attrs = {'class':'WoYOw'})
    for i in range(len(num)):
        try:
            review_list.append(num[i].find('span', attrs = {'class':'WoYOw'}).text)
        except:#사진만 있는 리뷰, 공백 발생
            review_list.append('')   
            
    # 유저 정보 -> 공백 발생 X
    user_names = soup.find_all('div', attrs = {'class':'_1vou-'})
    for user_name in user_names:
        user_list.append(user_name.text)
        url_list.append(thisurl)
        place_name.append(title)
            

In [18]:
# 수집된 데이터 개수 확인
len(place_name), len(review_list), len(url_list), len(user_list)

(165, 165, 165, 165)

In [19]:
# 각 리스트 중복 확인 (단순 확인 작업)
len(set(place_name)), len(set(review_list)), len(set(url_list)), len(set(user_list))

(3, 127, 3, 107)

In [20]:
# 수집한 리스트를 DataFrame으로 전환
data = {'place_name' : place_name,
        'user_name' : user_list,
        'review' : review_list,
        'url': url_list}
df = pd.DataFrame(data)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   place_name  165 non-null    object
 1   user_name   165 non-null    object
 2   review      165 non-null    object
 3   url         165 non-null    object
dtypes: object(4)
memory usage: 5.3+ KB


In [22]:
df

,place_name,user_name,review,url
0,스트릿 그릭요거트 카페,CJ65,커피도 맛있고 요거트볼도 맛있고 샌드위치도 맛있었어요! 평일 낮에 사람 별로 없을 ...,https://pcmap.place.naver.com/restaurant/19515...
1,스트릿 그릭요거트 카페,꽃길을 거닐다,그릭요거트 생과일볼과 애플시나몬 요거트볼 둘 다 달지 않고 담백하고 깔끔하니 맛있어...,https://pcmap.place.naver.com/restaurant/19515...
2,스트릿 그릭요거트 카페,kiri,샐러드를 먹는 방식이 특이하네용~ 샐러드의 이케아 효과?ㅎㅎ 창 밖에 나무 보여서 ...,https://pcmap.place.naver.com/restaurant/19515...
3,스트릿 그릭요거트 카페,쏘혜님,아몬드라떼 첨 마셔보는데 꼬숩고 맛있네요!\n주문시에 머그/일회용 등 선택할 수 있...,https://pcmap.place.naver.com/restaurant/19515...
4,스트릿 그릭요거트 카페,das****,지나가다가 들렸는데 진짜 분위기 맛집ㅠㅠ 곳곳마다 깨알소품들이랑 노래며 인테리어까지...,https://pcmap.place.naver.com/restaurant/19515...
...,...,...,...,...
160,모노커피,ku****,커피 맛있어요~~,https://pcmap.place.naver.com/restaurant/12816...
161,모노커피,ku****,커피 넘 맛있어여,https://pcmap.place.naver.com/restaurant/12816...
162,모노커피,미니3695,좋아요,https://pcmap.place.naver.com/restaurant/12816...
163,모노커피,위티브,감사합니다,https://pcmap.place.naver.com/restaurant/12816...


In [23]:
df.to_csv("naver_place_final.csv", encoding = 'utf-8-sig')

In [ ]:
#user 이름과 review 내용이 같다면 삭제
#df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index = True)
#df